# Merging School Locations with Tournament Data

The shorthand/common name for schools listed in the tournament data is not detailed enough to properly geocode, so schools must be associated with their respective cities and states. Other school metadata, such as team name/mascot and conference, will be useful for visualization and aggregation as well. Datasets for the top-seeded tournament teams and all NCAA D1 institutions will be cleaned and merged.

Source: [Wikipedia List of NCAA D1 Institutions](https://en.wikipedia.org/wiki/List_of_NCAA_Division_I_institutions)

## Import packages and raw data

Pandas will again be used to manipulate the data frames. Both the tournament data and D1 master list are imported.

In [1]:
import pandas as pd

In [2]:
# import full tournament data and check table
tourneyData = pd.read_csv('../data/cleaned/mm-85-19-cleaned.csv')
tourneyData

,seed,school,site,year,id
0,1.0,Duke,"Columbia, SC",2019.0,20190
1,1.0,Gonzaga,"Salt Lake City, UT",2019.0,20191
2,1.0,UNC,"Columbus, OH",2019.0,20192
3,1.0,Virginia,"Columbia, SC",2019.0,20193
4,2.0,Michigan State,"Des Moines, IA",2019.0,20194
...,...,...,...,...,...
1115,NaN,NaN,"Albuquerque, NM",NaN,1985555
1116,NaN,NaN,"Hartford, CT",NaN,1985556
1117,NaN,NaN,"Tulsa, OK",NaN,1985557
1118,NaN,NaN,"Dayton, OH",NaN,1985558


In [3]:
# import d1 master list and check
allSchools = pd.read_csv('../data/raw/d1-master-list.csv')
allSchools

,school,common_name,team,city,state,type,conference
0,Abilene Christian University,Abilene Christian,Wildcats,Abilene,Texas,Private/Churches of Christ,Southland Conference
1,University of Akron,Akron,Zips,Akron,Ohio,State,Mid-American Conference
2,University of Alabama,Alabama,Crimson Tide,Tuscaloosa,Alabama,State,Southeastern Conference
3,Alabama Agricultural and Mechanical University,Alabama A&M,Bulldogs and Lady Bulldogs,Huntsville,Alabama,State,Southwestern Athletic Conference
4,University of Alabama at Birmingham,UAB,Blazers,Birmingham,Alabama,State,Conference USA
...,...,...,...,...,...,...,...
345,Wright State University,Wright State,Raiders,Fairborn,Ohio,State,Horizon League
346,University of Wyoming,Wyoming,Cowboys and Cowgirls,Laramie,Wyoming,State,Mountain West Conference
347,Xavier University,Xavier,Musketeers,Cincinnati,Ohio,Private/Catholic,Big East Conference
348,Yale University,Yale,Bulldogs,New Haven,Connecticut,Private/Non-Sectarian,Ivy League


## Extract unique schools from tournament data

Though there are 16 top seeded teams for each tournament year, many of the same teams are consistently top ranked and are included many times thoroughout the tournament dataset. By finding all unique schools in the dataset, the shorthand names can be tied to the D1 master dataset.

In [4]:
# find all unique schools in data set, format as dataframe, check
uniqueSchools = pd.DataFrame({'school': list(tourneyData.school.unique())})
uniqueSchools

,school
0,Duke
1,Gonzaga
2,UNC
3,Virginia
4,Michigan State
...,...
83,DePaul
84,TCU
85,VCU
86,Loyola (IL)


## Clean up master D1 dataset

Many of the citations from Wikipedia are still visible in the master D1 dataset. Not only are they an eyesore, but they will disrupt finding equality between fields/strings. They will be removed by using regular expressions to replace any text contained within brackets (and the brackets themselves) with an empty string.

Many of the teams also include separate gendered mascots for the men's and women's teams. This is unnecessary for this project, so any extraneous mascots will be removed.

### Remove all citations brackets

In [5]:
# loop through all columns in schools master list, replace any text between brackets with empty string
for cols in allSchools:
    allSchools[cols] = allSchools[cols].str.replace(r"\[.*\]", '')

# check - can see that [h] was removed from the final row conference column (Horizon League[h])
allSchools

,school,common_name,team,city,state,type,conference
0,Abilene Christian University,Abilene Christian,Wildcats,Abilene,Texas,Private/Churches of Christ,Southland Conference
1,University of Akron,Akron,Zips,Akron,Ohio,State,Mid-American Conference
2,University of Alabama,Alabama,Crimson Tide,Tuscaloosa,Alabama,State,Southeastern Conference
3,Alabama Agricultural and Mechanical University,Alabama A&M,Bulldogs and Lady Bulldogs,Huntsville,Alabama,State,Southwestern Athletic Conference
4,University of Alabama at Birmingham,UAB,Blazers,Birmingham,Alabama,State,Conference USA
...,...,...,...,...,...,...,...
345,Wright State University,Wright State,Raiders,Fairborn,Ohio,State,Horizon League
346,University of Wyoming,Wyoming,Cowboys and Cowgirls,Laramie,Wyoming,State,Mountain West Conference
347,Xavier University,Xavier,Musketeers,Cincinnati,Ohio,Private/Catholic,Big East Conference
348,Yale University,Yale,Bulldogs,New Haven,Connecticut,Private/Non-Sectarian,Ivy League


### Rename repetitious 'school' column names

In [6]:
# rename school column as fullnames
allSchools.rename(index=str, columns={'school': 'school_full_name'}, inplace=True)
allSchools

,school_full_name,common_name,team,city,state,type,conference
0,Abilene Christian University,Abilene Christian,Wildcats,Abilene,Texas,Private/Churches of Christ,Southland Conference
1,University of Akron,Akron,Zips,Akron,Ohio,State,Mid-American Conference
2,University of Alabama,Alabama,Crimson Tide,Tuscaloosa,Alabama,State,Southeastern Conference
3,Alabama Agricultural and Mechanical University,Alabama A&M,Bulldogs and Lady Bulldogs,Huntsville,Alabama,State,Southwestern Athletic Conference
4,University of Alabama at Birmingham,UAB,Blazers,Birmingham,Alabama,State,Conference USA
...,...,...,...,...,...,...,...
345,Wright State University,Wright State,Raiders,Fairborn,Ohio,State,Horizon League
346,University of Wyoming,Wyoming,Cowboys and Cowgirls,Laramie,Wyoming,State,Mountain West Conference
347,Xavier University,Xavier,Musketeers,Cincinnati,Ohio,Private/Catholic,Big East Conference
348,Yale University,Yale,Bulldogs,New Haven,Connecticut,Private/Non-Sectarian,Ivy League


### Remove gender-specific mascots from team list

In [7]:
# remove any gender-specific mascots from team list
# split string on 'and', creates list of value before and after split
splitMascots = allSchools.team.str.split(' and')

# save only the first value before 'and' as new team column
allSchools.team = [mascot[0] for mascot in splitMascots]

# remove any left over whitespace
allSchools.team = allSchools.team.str.strip()

# check
allSchools.team

0          Wildcats
1              Zips
2      Crimson Tide
3          Bulldogs
4           Blazers
           ...     
345         Raiders
346         Cowboys
347      Musketeers
348        Bulldogs
349        Penguins
Name: team, Length: 350, dtype: object

## Attempt merge to find any school name mismatches

While both datasets have school common names, there are often several shorthands for each school and it is not clear which ones will match. Attempting a left-join will illuminate failures by matching them with NAs.

In [8]:
# try to merge data sets to find values with different common names
mergeTest = pd.merge(uniqueSchools, allSchools, how='left', left_on='school', right_on='common_name')

# pull out any merge failures - any unmerged school will be NA, check
fails = mergeTest[mergeTest['common_name'].isna()]
fails

,school,school_full_name,common_name,team,city,state,type,conference
2,UNC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,Pitt,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,St. Joseph's,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,St. John's (NY),NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,Loyola (IL),NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Rename mismatched schools

Because there are only 5 unmatched schools, they can easily be renamed manually. All extra NA values will also be dropped.

In [9]:
# rename North Carolina, Pittsburgh, Saint Joseph's, St. John's, Loyola-Chicago
uniqueSchools[uniqueSchools.school == 'UNC'] = 'North Carolina'
uniqueSchools[uniqueSchools.school == 'Pitt'] = 'Pittsburgh'
uniqueSchools[uniqueSchools.school == "St. Joseph's"] = "Saint Joseph's"
uniqueSchools[uniqueSchools.school == "St. John's (NY)"] = "St. John's"
uniqueSchools[uniqueSchools.school == 'Loyola (IL)'] = 'Loyola–Chicago'

# drop all na
uniqueSchools = uniqueSchools.dropna()

# check
uniqueSchools

,school
0,Duke
1,Gonzaga
2,North Carolina
3,Virginia
4,Michigan State
...,...
82,La Salle
83,DePaul
84,TCU
85,VCU


## Merge cleaned datasets

In [10]:
mergedSchools = pd.merge(uniqueSchools, allSchools, how='left', left_on='school', right_on='common_name')
mergedSchools

,school,school_full_name,common_name,team,city,state,type,conference
0,Duke,Duke University,Duke,Blue Devils,Durham,North Carolina,Private/Non-sectarian,Atlantic Coast Conference
1,Gonzaga,Gonzaga University,Gonzaga,Bulldogs,Spokane,Washington,Private/Catholic,West Coast Conference
2,North Carolina,University of North Carolina at Chapel Hill,North Carolina,Tar Heels,Chapel Hill,North Carolina,State,Atlantic Coast Conference
3,Virginia,University of Virginia,Virginia,Cavaliers,Charlottesville,Virginia,State,Atlantic Coast Conference
4,Michigan State,Michigan State University,Michigan State,Spartans,East Lansing,Michigan,State,Big Ten Conference
...,...,...,...,...,...,...,...,...
82,La Salle,La Salle University,La Salle,Explorers,Philadelphia,Pennsylvania,Private/Catholic,Atlantic 10 Conference
83,DePaul,DePaul University,DePaul,Blue Demons,Chicago,Illinois,Private/Catholic,Big East Conference
84,TCU,Texas Christian University,TCU,Horned Frogs,Fort Worth,Texas,Private/Disciples of Christ,Big 12 Conference
85,VCU,Virginia Commonwealth University,VCU,Rams,Richmond,Virginia,State,Atlantic 10 Conference


## Test for failures

None here! Good to move forward. :)

In [11]:
mergeFails = mergedSchools[mergedSchools['common_name'].isna()]
mergeFails

,school,school_full_name,common_name,team,city,state,type,conference


## Fix school mismatches in the original dataframe

Though the mismatched common names in the `uniqueNames` list were fixed, the full `tourneyData` dataframe needs to be fixed as well. 

In [12]:
# replace ALL mismatched data
tourneyData = tourneyData.replace(to_replace='UNC', value='North Carolina')
tourneyData = tourneyData.replace(to_replace='Pitt', value='Pittsburgh') 
tourneyData = tourneyData.replace(to_replace="St. Joseph's", value="Saint Joseph's") 
tourneyData = tourneyData.replace(to_replace="St. John's (NY)", value="St. John's") 
tourneyData = tourneyData.replace(to_replace='Loyola (IL)', value='Loyola–Chicago')

# check
tourneyData

,seed,school,site,year,id
0,1.0,Duke,"Columbia, SC",2019.0,20190
1,1.0,Gonzaga,"Salt Lake City, UT",2019.0,20191
2,1.0,North Carolina,"Columbus, OH",2019.0,20192
3,1.0,Virginia,"Columbia, SC",2019.0,20193
4,2.0,Michigan State,"Des Moines, IA",2019.0,20194
...,...,...,...,...,...
1115,NaN,NaN,"Albuquerque, NM",NaN,1985555
1116,NaN,NaN,"Hartford, CT",NaN,1985556
1117,NaN,NaN,"Tulsa, OK",NaN,1985557
1118,NaN,NaN,"Dayton, OH",NaN,1985558


## Merge metadata with entire tournament dataframe

Now that there is a dataframe with all of the metadata for each unique school, it can be merged back into the original `tourneyData` dataframe.

In [13]:
# merge all datasets
mergedTourney = pd.merge(tourneyData, mergedSchools, how='left', left_on="school", right_on="common_name")

# check
mergedTourney

,seed,school_x,site,year,id,school_y,school_full_name,common_name,team,city,state,type,conference
0,1.0,Duke,"Columbia, SC",2019.0,20190,Duke,Duke University,Duke,Blue Devils,Durham,North Carolina,Private/Non-sectarian,Atlantic Coast Conference
1,1.0,Gonzaga,"Salt Lake City, UT",2019.0,20191,Gonzaga,Gonzaga University,Gonzaga,Bulldogs,Spokane,Washington,Private/Catholic,West Coast Conference
2,1.0,North Carolina,"Columbus, OH",2019.0,20192,North Carolina,University of North Carolina at Chapel Hill,North Carolina,Tar Heels,Chapel Hill,North Carolina,State,Atlantic Coast Conference
3,1.0,Virginia,"Columbia, SC",2019.0,20193,Virginia,University of Virginia,Virginia,Cavaliers,Charlottesville,Virginia,State,Atlantic Coast Conference
4,2.0,Michigan State,"Des Moines, IA",2019.0,20194,Michigan State,Michigan State University,Michigan State,Spartans,East Lansing,Michigan,State,Big Ten Conference
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,NaN,NaN,"Albuquerque, NM",NaN,1985555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1116,NaN,NaN,"Hartford, CT",NaN,1985556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117,NaN,NaN,"Tulsa, OK",NaN,1985557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1118,NaN,NaN,"Dayton, OH",NaN,1985558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Clean data before saving

Remove extra school columns.

In [14]:
# df.rename(index=str, columns={"Ctyname": "county", "Stname": "state"}, inplace=True)
mergedTourney.rename(index=str, columns={"school_x": "school_common_name"}, inplace=True)

# remove repeated common name columns
mergedTourney = mergedTourney.drop(['school_y', 'common_name'], axis=1)
mergedTourney

,seed,school_common_name,site,year,id,school_full_name,team,city,state,type,conference
0,1.0,Duke,"Columbia, SC",2019.0,20190,Duke University,Blue Devils,Durham,North Carolina,Private/Non-sectarian,Atlantic Coast Conference
1,1.0,Gonzaga,"Salt Lake City, UT",2019.0,20191,Gonzaga University,Bulldogs,Spokane,Washington,Private/Catholic,West Coast Conference
2,1.0,North Carolina,"Columbus, OH",2019.0,20192,University of North Carolina at Chapel Hill,Tar Heels,Chapel Hill,North Carolina,State,Atlantic Coast Conference
3,1.0,Virginia,"Columbia, SC",2019.0,20193,University of Virginia,Cavaliers,Charlottesville,Virginia,State,Atlantic Coast Conference
4,2.0,Michigan State,"Des Moines, IA",2019.0,20194,Michigan State University,Spartans,East Lansing,Michigan,State,Big Ten Conference
...,...,...,...,...,...,...,...,...,...,...,...
1115,NaN,NaN,"Albuquerque, NM",NaN,1985555,NaN,NaN,NaN,NaN,NaN,NaN
1116,NaN,NaN,"Hartford, CT",NaN,1985556,NaN,NaN,NaN,NaN,NaN,NaN
1117,NaN,NaN,"Tulsa, OK",NaN,1985557,NaN,NaN,NaN,NaN,NaN,NaN
1118,NaN,NaN,"Dayton, OH",NaN,1985558,NaN,NaN,NaN,NaN,NaN,NaN


## Write to CSV

In [16]:
mergedTourney.to_csv('../data/cleaned/tourney-metadata.csv', index=False)

,school_full_name,common_name,team,city,state,type,conference
0,Abilene Christian University,Abilene Christian,Wildcats,Abilene,Texas,Private/Churches of Christ,Southland Conference
1,University of Akron,Akron,Zips,Akron,Ohio,State,Mid-American Conference
2,University of Alabama,Alabama,Crimson Tide,Tuscaloosa,Alabama,State,Southeastern Conference
3,Alabama Agricultural and Mechanical University,Alabama A&M,Bulldogs,Huntsville,Alabama,State,Southwestern Athletic Conference
4,University of Alabama at Birmingham,UAB,Blazers,Birmingham,Alabama,State,Conference USA
...,...,...,...,...,...,...,...
345,Wright State University,Wright State,Raiders,Fairborn,Ohio,State,Horizon League
346,University of Wyoming,Wyoming,Cowboys,Laramie,Wyoming,State,Mountain West Conference
347,Xavier University,Xavier,Musketeers,Cincinnati,Ohio,Private/Catholic,Big East Conference
348,Yale University,Yale,Bulldogs,New Haven,Connecticut,Private/Non-Sectarian,Ivy League


In [17]:
# save cleaned all schools data
allSchools.to_csv('../data/cleaned/d1-schools-cleaned.csv', index=False)